In [ ]:
class Template():
    def __init__(self, hard_fitlers, soft_filters, failscore=0):
        self.hard_fitlers = hard_fitlers
        self.soft_filters = soft_filters
        self.failscore = failscore
        self.dataset = None
        
    def __call__(self, mol):
        
        output = maybe_parallel(self.eval_mol, mol)
            
        return output
        
    def eval_mol(self, mol):
        mol = to_mol(mol)
        hardpass = self.hard_fitlers(mol)
        if hardpass:
            score = self.soft_filters(mol)
        else:
            score = self.failscore
            
        return [hardpass, score]
    
    def process_smile(self, smile):
        hardpass, score = self.eval_mol(to_mol(smile))
        return [hardpass, to_smile(smile), score]

    def load_data(self, smiles):
        
        data = maybe_parallel(self.process_smile, smiles)
        data = [i[1:] for i in data if i[0]]
        
        new_dataset = pd.DataFrame(data, columns=['smiles', 'score'])
        
        if self.dataset is not None:
            self.dataset = pd.concat([self.dataset, new_dataset])
            self.dataset.drop_duplicates(subset='smiles', inplace=True)
            self.dataset.reset_index(inplace=True, drop=True)
        else:
            self.dataset = new_dataset
            
    def sample_data(self, n, seed=None):
        if seed is None:
            seed = np.random.seed()
            
        if self.dataset is not None:
            return self.dataset.sample(n=n, random_state=seed).reset_index(drop=True)
        else:
            return []
        
    def sample_smile(self, n):
        data = self.sample_data(n)
        if type(data)==pd.DataFrame:
            data = list(data.smiles.values)
        return data
        
    def __repr__(self):
        
        hr = self.hard_fitlers.__repr__().replace('\t', '\t\t')
        sr = self.soft_filters.__repr__().replace('\t', '\t\t')
        rep_str = 'Template\n\t' + hr + '\n\t' + sr
        return rep_str

In [ ]:
temp = Template(hf, sf, -5)

In [ ]:
temp

Template
	Hard Filter:
		molwt (150, 350)
		Structure filter, criteria: any, exclude: True
	Soft Filter:
		molwt (150, 350)
		Structure filter, criteria: any, exclude: True

In [ ]:
df = pd.read_csv('/Users/karl/Desktop/bindingDB/filtered_ligands.smi', header=None, sep='\t')
df.columns = ['smiles', 'name']

In [ ]:
df.head()

,smiles,name
0,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,"6-[(4R,5S,6S,7R)-4,7-dibenzyl-3-(5-carboxypent..."
1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b..."
2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy..."
3,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy..."
4,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy..."


In [ ]:
test = temp(df.smiles.values[:10000])

In [ ]:
temp.load_data(df.smiles.values[:10000])

In [ ]:
frag_df = pd.read_csv('fragments.csv')

In [ ]:
frag_df.head()

,smiles,attachments,molwt,rings,nrot
0,*C(=O)NC(CC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(C(=O...,3,399.151589,0,13
1,*CC(C)CN(CC(O)C(*)*)S(=O)(=O)c1ccc(CO)cc1,3,298.111304,1,9
2,*CSCC(=O)C(*)NC(=O)c1ccc(CN(*)S(=O)(=O)c2ccc(*...,4,438.001399,2,11
3,*C(CC(=O)O)NC(=O)c1ccc(CNCc2ccc(O)c(C(=O)O)c2)s1,1,377.080732,2,9
4,*c1cc(C)c(C(=O)NC(*)C=O)s1,2,181.019749,1,3


In [ ]:
class Block():
    def __init__(self, template, links, name, subblocks=[]):
        self.template = template
        self.links = links
        self.name = name
        self.subblocks = subblocks
        self.sublinks = []
        
    def update_links(self):
        # updates self.sublinks based on self.subblocks
        pass
        
    def match_fragment(self, fragment):
        # determine if fragment matches block link pattern
        pass
    
    def match_fragment_recursive(self, fragment):
        # recursively match fragment to all subblocks
        if self.match_fragment(fragment):
            output = True
        else:
            output = False
            for block in self.subblocks:
                if block.match_fragment_recursive(fragment):
                    output = True
                    
        return output
    
    def eval_mol(self, mol):
        mol = to_mol(mol)
        smile = to_smile(mol)
        
        if self.match_fragment(smile):
            hardpass, score = self.template(mol)
        else:
            hardpass = False
            score = self.template.failscore
            
        return [hardpass, score]
    
    def add_mapping(self, fragment):
        # converts unmapped fragment to mapped fragment
        pass
    
    def remove_mapping(self, fragment):
        # converts mapped fragment to unmapped fragment
        pass

    def shuffle_mapping(sef, fragment):
        # shuffles mapping of attachment points
        pass
    
    def check_num_links(self, fragment):
        # checks if the fragment has the same number of attachments as defined in self.links
        pass
    
    def process_fragment(self, fragment):
        # checks if fragment has correct number of attachments and maps fragment
        
        if self.check_num_links(fragment):
            fragment = self.add_mapping(fragment)
            attachment_pass = True
        else:
            attachment_pass = False
            
        return [attachment_pass, fragment]
    
    def load_data(self, fragments, recurse=False):
        # checks fragment attachments, then sends to template `load_data`
        # optionally recursive
        if recurse:
            for b in self.subblocks:
                b.load_data(fragments, recurse=True)
        
        fragments = maybe_parallel(self.process_fragment, fragments)
        fragments = [i[1] for i in fragments if i[0]]
        self.template.load_data(fragments)
            
    def sample_data(self, n):
        return self.template.sample_data(n)
    
    def decompose_fragments(self, fragment_string):
        # decomposes a string of multiple fragments into a list of single fragments
        pass
    
    def join_fragments(self, fragment_list):
        # joins list of fragments into single string
        pass
    
    def fuse_fragments(self, fragment_string):
        # fuses fragment string into single output
        pass
    
    def join_and_fuse(self, fragment_list):
        return self.fuse_fragments(self.join_fragments(fragment_list))
    
    def recurse_fragments(self, fragment):
        # recursively break down fragments, route to subblocks, fuse and evaluate
        pass
        
    def __repr__(self):
        
        rep_str = f'Block {self.name}: {self.links}\n\t' + '\n\t'.join(self.template.__repr__().split('\n'))
        
        if self.subblocks:
            rep_str += '\n'
            for b in self.subblocks:
                rep_str += '\n\t' + '\n\t'.join(b.__repr__().split('\n'))
                
        return rep_str
    
    
class MolBlock(Block):
    def __init__(self, template, links, name, subblocks=[]):
        super().__init__(template, links, name, subblocks=subblocks)
        
        self.update_links()
        self.pattern = re.compile('\[(.*?)\*:(.*?)]') # identifies isotope and map num
    
    def update_links(self):
        for b in self.subblocks:
            self.sublinks.append(b.links)
            for sl in b.sublinks:
                self.sublinks.append(sl)

    def match_fragment(self, fragment, do_map=True, links=None):
        # determine if fragment matches block link pattern
        
        if self.check_num_links(fragment):
            if do_map:
                self.add_mapping(fragment)

            if links is None:
                links = self.links

            pattern_matches = self.pattern.findall(fragment) # find isotope/map num pairs

            if set(pattern_matches) == set(links):
                link_match = True
            else:
                link_match = False
        else:
            link_match = False
        
        return link_match
            
    def add_mapping(self, fragment, links=None):
        pattern_matches = self.pattern.findall(fragment)
        
        if len(pattern_matches) == len(self.links):
            # already mapped
            output = fragment
        else:
            if len(pattern_matches) > 0:
                # partially mapped, something went wrong
                fragment = self.remove_mapping(fragment)
        
            if links is None:
                links = list(self.links)
                random.shuffle(links)
                
            new_fragment = ''
            link_count = 0
            for s in fragment:
                if s=='*':
                    isotope, map_num = links[link_count]
                    s = f'[{isotope}*:{map_num}]'
                    link_count += 1
            
                new_fragment += s
                
            output = new_fragment
            
        return output
    
    def remove_mapping(self, fragment):
        current_mapping = self.pattern.findall(fragment)
        
        for cm in current_mapping:
            isotope, map_num = cm
            fragment = fragment.replace(f'[{isotope}*:{map_num}]', '*')
            
        return fragment
    
    def shuffle_mapping(self, fragment):
        current_mapping = self.pattern.findall(fragment)
        new_mapping = list(current_mapping)
        random.shuffle(new_mapping)
        
        fragment = self.remove_mapping(fragment)
        fragment = self.add_mapping(fragment, links=new_mapping)
        return fragment
    
    def check_num_links(self, fragment):
        return fragment.count('*') == len(self.links)
    
    def sample_smile(self, n):
        return self.template.sample_smile(n)
    
    def decompose_fragments(self, fragment_string):
        return fragment_string.split('.')
    
    def join_fragments(self, fragment_list):
        return '.'.join(fragment_list)
    
    def fuse_fragments(self, fragment_string):
        
        new_smile = fuse_on_atom_mapping(fragment_string)
        
        if '[*' in new_smile: #lost isotope
            new_smile = new_smile.replace('[*', '[0*')
            
        return new_smile
    
    def recurse_fragments(self, fragments):

        output_dicts = []
        total_pass = []
        total_score = 0.

        if type(fragments) == str:
            fragments = [fragments]

        fragments = [self.decompose_fragments(i) for i in fragments]
        fragments = [item for sublist in fragments for item in sublist]

        if self.subblocks:
            new_fragments = []

            unrouted = list(fragments) # copy list

            for sb in self.subblocks:
                routed = [i for i in unrouted if sb.match_fragment_recursive(i)]
                unrouted = [i for i in unrouted if not i in routed]

                if routed:
                    r_fused, r_pass, r_score, subdicts = sb.recurse_fragments(routed)
                    new_fragments.append(r_fused)
                    total_pass.append(r_pass)
                    total_score += r_score
                    output_dicts += subdicts

                if isinstance(sb, ConstantBlock):
                    new_fragments.append(sb.smile)

            fragments = new_fragments + unrouted

        joined_fragments = self.join_fragments(fragments)
        fused = self.fuse_fragments(joined_fragments)

        frag_pass, frag_score = self.eval_mol(fused)
        total_pass.append(frag_pass)
        total_score += frag_score

        total_pass = all(total_pass)

        output_dict = {
            'block' : self.name,
            'fused' : fused,
            'fragments' : fragments,
            'block_pass' : frag_pass,
            'block_score' : frag_score,
            'all_pass' : total_pass,
            'all_score' : total_score
        }

        output_dicts.append(output_dict)

        return fused, total_pass, total_score, output_dicts
    
    def __repr__(self):
        
        rep_str = f'Block {self.name}: {self.links}\n\t' + '\n\t'.join(self.template.__repr__().split('\n'))
        
        if self.subblocks:
            rep_str += '\n'
            for b in self.subblocks:
                rep_str += '\n\t' + '\n\t'.join(b.__repr__().split('\n'))
                
        return rep_str
    

In [ ]:
class ConstantBlock():
    def __init__(self, constant, name):
        self.constant = constant
        self.name = name
        self.links = []
        self.subblocks = []
        self.sublinks = []
        
    def match_fragment(self, fragment):
        return False
    
    def match_fragment_recursive(self, fragment):
        return False

    def load_data(self, fragments, recurse=False):
        pass
            
    def sample_data(self, n):
        return pd.DataFrame([self.constant]*n, columns=['smiles'])
    
    def __repr__(self):
        
        rep_str = f'Constant Block: {self.constant}'
        
        return rep_str
    

class ConstantMolBlock(ConstantBlock):
    def __init__(self, smile, name):
        super().__init__(smile, name)
        self.smile = Chem.CanonSmiles(smile)
        if '[*' in self.smile:
            self.smile = self.smile.replace('[*', '[0*')
        self.pattern = re.compile('\[(.*?)\*:(.*?)]')
        self.links = self.pattern.findall(smile)
    
    def sample_smile(self, n):
        return [self.smile]*n
    
    def __repr__(self):
        
        rep_str = f'Constant Block: {self.smile}'
        
        return rep_str

In [ ]:
# scheme - constant scaffold, two variable r groups

# scaffod
scaffold_smile = 'c1nc2c([0*:2])cncc2cc1[0*:1]'
scaffold_block = ConstantMolBlock(scaffold_smile, 'scaffold')

# R1, must have ring, be between 50-250 g/mol. must have 1 ring. ideally less thn 100-200 g/mol

r1_template = Template(
                    HardFilters([
                        MolWtFilter(50, 250, None),
                        RingFilter(1,1,None)
                    ]),
                    SoftFilters([
                        MolWtFilter(100,200,1),
                    ]),
                    failscore = -1)
r1_block = MolBlock(r1_template, [('1', '1')], 'r1')


# R1, must have no rings, be between 0-200 g/mol. must have 0 rings. ideally less thn 50-150 g/mol

r2_template = Template(
                    HardFilters([
                        MolWtFilter(0, 200, None),
                        RingFilter(0,0,None)
                    ]),
                    SoftFilters([
                        MolWtFilter(50,150,1),
                    ]),
                    failscore = -1)
r2_block = MolBlock(r2_template, [('1', '2')], 'r2')


# full compound, must be between 200 and 550 g/mol

full_template = Template(
                    HardFilters([
                        MolWtFilter(200, 550, None)
                    ]),
                    SoftFilters([
                        MolWtFilter(200,550,1),
                    ]),
                    failscore = -1)

main_block = MolBlock(full_template, [], 'full_molecue', subblocks=[scaffold_block, r1_block, r2_block])

In [ ]:
main_block.load_data(df.smiles.values[:10000], recurse=True)

In [ ]:
main_block.load_data(frag_df.smiles.values[:100000], recurse=True)

In [ ]:
fragment_string = '[1*:2]C(=O)NC(CO)CC(C)C.[1*:1]S(=O)(=O)NCc1ccc(O)c(C(=O)O)c1'

In [ ]:
main_block.fuse_fragments(fragment_string+'.'+scaffold_block.smile)

'CC(C)CC(CO)NC(=O)c1cncc2cc(S(=O)(=O)NCc3ccc(O)c(C(=O)O)c3)cnc12'

In [ ]:
main_block.match_fragment('[1*:2]C(=O)NC(CO)CC(C)C')

False

In [ ]:
main_block.match_fragment_recursive('[1*:2]C(=O)NC(CO)CC(C)C')

True

In [ ]:
out = main_block.recurse_fragments('[1*:2]C(=O)NC(CO)CC(C)C.[1*:1]S(=O)(=O)NCc1ccc(O)c(C(=O)O)c1')

In [ ]:
out[0]

'CC(C)CC(CO)NC(=O)c1cncc2cc(S(=O)(=O)NCc3ccc(O)c(C(=O)O)c3)cnc12'

In [ ]:
out[1]

True

In [ ]:
out[2]

2.0

In [ ]:
# scheme - constant scaffold, two variable r groups. one r group has a ring with two attachments, 
# one constant and one variabe

# scaffod
scaffold_smile = 'c1nc2c([0*:2])cncc2cc1[0*:1]'
scaffold_block = ConstantMolBlock(scaffold_smile, 'scaffold')

# R1 has 3 groups - constant carbonyl, variable ring, variabe ring attachment

r1_carbonyl_block = ConstantMolBlock('C(O)(=O)[1*:4]', 'carbonyl')

r1_ring_substitution_tempate = Template(
                                    HardFilters([
                                        RotBondFilter(0, 3, None),
                                        RingFilter(0,0,None)
                                    ]),
                                    SoftFilters([
                                      RotBondFilter(0, 2, 1)
                                    ]),
                                    failscore = -1)

r1_ring_substitution_block = MolBlock(r1_ring_substitution_tempate, [('1', '3')], 'r1 ring substitution')

r1_ring_template = Template(
                            HardFilters([
                                RingFilter(1,1,None),
                                RotBondFilter(0,1, None)
                            ]),
                            SoftFilters([
                                RingFilter(1,1,1)
                            ]))
r1_ring_block = MolBlock(r1_ring_template, [('0', '3'), ('0', '4'), ('1', '2')], 'r1_ring')


r1_full_group_template = Template(
                            HardFilters([
                                MolWtFilter(50, 250, None)
                            ]),
                            SoftFilters([
                                MolWtFilter(100, 200, 1)
                            ]))
r1_block = MolBlock(r1_full_group_template, [('1', '2')], 'r1_full', 
                     subblocks=[r1_carbonyl_block, r1_ring_substitution_block, r1_ring_block])


# R1, must have no rings, be between 0-200 g/mol. must have 0 rings. ideally less thn 50-150 g/mol

r2_template = Template(
                    HardFilters([
                        MolWtFilter(0, 200, None),
                        RingFilter(0,0,None)
                    ]),
                    SoftFilters([
                        MolWtFilter(50,150,1),
                    ]),
                    failscore = -1)
r2_block = MolBlock(r2_template, [('1', '1')], 'r2')


# full compound, must be between 200 and 550 g/mol

full_template = Template(
                    HardFilters([
                        MolWtFilter(200, 550, None),
                        StructureFilter(['[#6](-[#8])(=[#8])-[*]',
                             '[#6]1:[#7]:[#6]2:[#6](-[*]):[#6]:[#7]:[#6]:[#6]:2:[#6]:[#6]:1-[*]'
                             ], 1, criteria='all', exclude=False)
                    ]),
                    SoftFilters([
                        MolWtFilter(200,550,1),
                    ]),
                    failscore = -1)

main_block = MolBlock(full_template, [], 'full_molecule', subblocks=[scaffold_block, r1_block, r2_block])

In [ ]:
main_block.load_data(df.smiles.values[:200000], recurse=False)

In [ ]:
main_block.load_data(frag_df.smiles.values[:200000], recurse=True)

In [ ]:


def sample_leaf_nodes(block, include_constant=False):
    
    if block.subblocks:
        output = [sample_leaf_nodes(i, include_constant=include_constant) for i in block.subblocks]
    else:
        if not isinstance(block, ConstantBlock):
            output = block.sample_smile(1)
        else:
            if include_constant:
                output = block.sample_smile(1)
            else:
                output = None
    
    if type(output) == list:
        output = flatten_recursive(output)
        output = [i for i in output if i is not None]
            
    return output

def sample_leaf_nodes_n(n, block, include_constant=False):
    output = [sample_leaf_nodes(block, include_constant=include_constant) for i in range(n)]
    return output

In [ ]:
frags = maybe_parallel(sample_leaf_nodes_n, [1]*10000, block=main_block, include_constant=False)

In [ ]:
frags = [item for sublist in frags for item in sublist]

In [ ]:
frags[0]

['[1*:3]CCCCC#N', '[0*:3]CC1C(O)C(O)C([1*:2])NC(=O)N1[0*:4]', '[1*:1]CCSC']

In [ ]:
fused = maybe_parallel(main_block.recurse_fragments, frags)

In [ ]:
fused_smiles = []
score_dicts = []
for item in fused:
    fused_smiles.append(item[0])
    score_dicts+= item[-1]

In [ ]:
fuse_df = pd.DataFrame(score_dicts)

In [ ]:
fuse_df.duplicated(subset='fused').mean()

0.63992

In [ ]:
fuse_df.drop_duplicates(subset='fused', inplace=True)

In [ ]:
fuse_df.reset_index(inplace=True, drop=True)

In [ ]:
fuse_df

,block,fused,fragments,block_pass,block_score,all_pass,all_score
0,r1 ring substitution,N#CCCCC[1*:3],[[1*:3]CCCCC#N],True,0.0,True,0.0
1,r1_ring,O=C1NC([1*:2])C(O)C(O)C(C[0*:3])N1[0*:4],[[0*:3]CC1C(O)C(O)C([1*:2])NC(=O)N1[0*:4]],False,0.0,False,0.0
2,r1_full,N#CCCCCCC1C(O)C(O)C([1*:2])NC(=O)N1C(=O)O,"[O=C(O)[1*:4], N#CCCCC[1*:3], O=C1NC([1*:2])C(...",False,0.0,False,0.0
3,r2,CSCC[1*:1],[[1*:1]CCSC],True,1.0,True,1.0
4,full_molecule,CSCCc1cnc2c(C3NC(=O)N(C(=O)O)C(CCCCCC#N)C(O)C3...,"[c1nc2c([0*:2])cncc2cc1[0*:1], N#CCCCCCC1C(O)C...",True,1.0,False,2.0
...,...,...,...,...,...,...,...
17999,r1_full,NC(CO)C1NC(=O)NC(C(=O)O)C1[1*:2],"[O=C(O)[1*:4], NC(CO)[1*:3], O=C1NC([0*:3])C([...",True,0.0,False,1.0
18000,full_molecule,CC(CCc1cnc2c(C3C(C(=O)O)NC(=O)NC3C(N)CO)cncc2c...,"[c1nc2c([0*:2])cncc2cc1[0*:1], NC(CO)C1NC(=O)N...",True,1.0,False,3.0
18001,r1_full,CSCCC1C(O)C(CC(=O)O)N(C)S(=O)(=O)N(C)C1[1*:2],"[O=C(O)[1*:4], CSCC[1*:3], CN1C(C[0*:4])C(O)C(...",False,0.0,False,1.0
18002,full_molecule,CSCCC1C(O)C(CC(=O)O)N(C)S(=O)(=O)N(C)C1c1cncc2...,"[c1nc2c([0*:2])cncc2cc1[0*:1], CSCCC1C(O)C(CC(...",True,1.0,False,3.0


In [ ]:
subfrags = pd.DataFrame(frags, columns=['f1', 'f2', 'f3'])

In [ ]:
subfrags.shape[0] - subfrags.duplicated().sum()

9988

In [ ]:
[i for i in fused if '*' in i[0]]

[]

In [ ]:
fuse_df[(fuse_df.block=='full_molecule') & (fuse_df.block_pass==True)].iloc[0].fragments

['c1nc2c([0*:2])cncc2cc1[0*:1]',
 'N#CCCCCCC1C(O)C(O)C([1*:2])NC(=O)N1C(=O)O',
 'CSCC[1*:1]']

In [ ]:
fuse_df[fuse_df.fused=='COC(=O)CCN1C([1*:2])C(O)C(O)C(C(=O)O)N(C)S1(=O)=O']

,block,fused,fragments,block_pass,block_score,all_pass,all_score


In [ ]:
r1_full_group_template('COC(=O)CCN1C([1*:2])C(O)C(O)C(C(=O)O)N(C)S1(=O)=O')

[False, 0]

In [ ]:
molwt(to_mol('COC(=O)CCN1C([1*:2])C(O)C(O)C(C(=O)O)N(C)S1(=O)=O'))

325.070561504

In [ ]:
class BlockTree():
    def __init__(self, head_block):
        self.head_block = head_block
        self.nodes = self.nodes_to_list(self.head_block)
        self.leaf_nodes = [i for i in self.nodes if not i.subblocks]
        self.live_leafs = [i for i in self.leaf_nodes if not isinstance(i, ConstantBlock)]
        self.node_dict = {i.name:i for i in self.nodes}
        
    def nodes_to_list(self, block):
        nodes = [block]
        if block.subblocks:
            for subblock in block.subblocks:
                nodes += self.nodes_to_list(subblock)
                
        return nodes
    
    def sample_leaf_nodes(self, include_constant=False):
        pass
    
    def combinatorial_sample(self, n_sample, max_n, include_constant=False):
        pass
    
    
class MolBlockTree(BlockTree):
    def __init__(self, head_block):
        super().__init__(head_block)
        
    def _sample_leaf_nodes(self, include_constant=False):
        if include_constant:
            leaf_nodes = self.leaf_nodes
        else:
            leaf_nodes = self.live_leafs
            
        output = []
        
        for node in leaf_nodes:
            output.append(node.sample_data(1))
        
        output = [list(i.smiles.values) for i in output]
            
        return output
    
    def sample_leaf_nodes(self, n, include_constant=False):
        return maybe_parallel(self._sample_leaf_nodes, [include_constant]*n)
    
    def combinatorial_sample(self, n_sample, max_n, include_constant=False):
        
        if include_constant:
            leaf_nodes = self.leaf_nodes
        else:
            leaf_nodes = self.live_leafs
            
        output = []
        
        for node in leaf_nodes:
            output.append(node.sample_data(n_sample))
        
        output = [list(set(i.smiles.values)) for i in output]
        it = itertools.product(*output)
        
        prods = []
        
        for i, p in enumerate(it):
            prods.append(p)
            if i>max_n:
                break
        
        return prods

In [ ]:
tree = MolBlockTree(main_block)